# Machine Learning Project

## Table of Contents
- [Import Data](#import-data)
- [Data Exploration](#data-exploration)
  - [Boolean Features](#boolean-features)
  - [Categorical Features](#categorical-features)
  - [Numerical Features](#numerical-features)
    - [Numerical Plots](#numerical-plots)
- [Pre-processing](#pre-processing)
  - [Functions](#functions)
  - [Missing Values](#missing-values)
  - [String Distance](#string-distance)

<a id="import-data"></a>
## Import Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# to calculate distance between strings
import nltk

In [ ]:
df = pd.read_csv('data/train.csv').set_index('carID')
df.head()

In [ ]:
df.info()

In [ ]:
num_duplicated_ids = df.index.duplicated().sum()
print(f'Number of duplicated carIDs: {num_duplicated_ids}')

#### Import Data Summary
- Dataset loaded successfully with `carID` as the index
- There are no duplicate entries in carID
- The dataset contains information about cars including both numerical features (price, mileage, tax, etc.) and categorical features (brand, model, transmission, etc.)
- Initial inspection shows multiple features that will require preprocessing:
  - Numerical features that need cleaning (negative values, outliers)
  - Categorical features that need standardization
  - Presence of missing values in several columns

<a id="data-exploration"></a>
## Data Exploration

<a id="boolean-features"></a>
### Boolean Features

In [ ]:
df['hasDamage'].value_counts(dropna=False)

#### Boolean Features Analysis

Key observations about `hasDamage` feature:
- Only contains binary values (0) and NaN
- No instances of value 1 found, suggesting potential data collection issues
- May indicate:
  - Cars with damage not being listed
  - System default setting of 0 for non-damaged cars
  - Incomplete damage assessment process
- Requires special handling in preprocessing:
  - Consider treating NaN as a separate category
  - Validate if 0 truly represents "no damage"
  - May need to be treated as a categorical rather than boolean feature

<a id="categorical-features"></a>
### Categorical Features

#### Check Categorical Features Consistency

In [ ]:
# List of categorical features
cat_cols = ['Brand', 'model', 'fuelType', 'transmission']

# Identify outlier examples in categorical features
cat_outliers_examples = {col: df[col].value_counts().tail(10).index for col in cat_cols}

# Display the outlier examples
pd.DataFrame(cat_outliers_examples)

#### Categorical Features Summary
- Initial analysis reveals significant data quality issues across all categorical columns
- No standardization in categorical features, with multiple variations of the same values (different spellings, capitalizations)
- Solution: We will implement string distance-based standardization using the `thefuzz` library to clean and standardize these features

<a id="numerical-features"></a>
### Numerical Features

In [ ]:
# Dict of numerical features
num_cols = {
    'price': 'continuous',
    'mileage': 'continuous',
    'tax': 'continuous',
    'mpg': 'continuous',
    'paintQuality%': 'continuous',
    'engineSize': 'continuous',
    'year': 'discrete',
    'previousOwners': 'discrete'
}

<a id="plots"></a>
#### Numerical Plots

In [ ]:
# Plot figures for numerical features and the target variable (price)
plt.figure(figsize=(16, 10))
for i, (col, var_type) in enumerate(num_cols.items(), 1):
    plt.subplot(4, 2, i)

    # Plot based on variable type
    if var_type == 'continuous':
        sns.histplot(data=df, x=col, kde=True, color="lightcoral", bins=30)
        plt.title(f"Distribution of {col}", fontsize=11)
    elif var_type == 'discrete':
        sns.countplot(data=df, x=col, color="lightcoral")
        plt.title(f"Distribution of {col}", fontsize=11)
        plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots for continuous numerical features and the target variable (price)
continuous_cols = [col for col, var_type in num_cols.items() if var_type == 'continuous']
plt.figure(figsize=(16, 10))
for i, col in enumerate(continuous_cols, 1):
    plt.subplot(3, 2, i)
    sns.boxplot(data=df, x=col, color="lightblue")
    plt.title(f"Distribution of {col}", fontsize=11)

plt.tight_layout()
plt.show()

#### Analysis of Numerical Distributions

Key observations from the plots:
- **Target Variable (Price)**:
  - Highly right-skewed distribution
  - Contains significant number of outliers in the upper range
  - Most cars are concentrated in the lower price range

- **Mileage**:
  - Right-skewed distribution
  - Large range from nearly new cars to high-mileage vehicles
  - Some outliers in upper range suggesting possible data entry errors

- **Tax**:
  - Multiple peaks suggesting different tax bands
  - Contains negative values which require investigation (possible tax benefits/rebates)
  - Large number of outliers on both ends of the distribution

- **MPG (Miles Per Gallon)**:
  - Approximately normal distribution with slight right skew
  - Some unrealistic extreme values that need cleaning
  - Reasonable median around typical car efficiency ranges

- **Paint Quality %**:
  - Contains values above 100% which are logically impossible
  - Left-skewed distribution suggesting optimistic ratings
  - Requires standardization to 0-100 range

- **Engine Size**:
  - Multi-modal distribution indicating common engine sizes
  - Some unusual patterns that need investigation
  - Contains outliers that may represent specialty vehicles

- **Year**:
  - Should be discrete but contains decimal values
  - Most cars are from recent years (2015-2023)
  - Some unrealistic values (pre-1950 or post-2025)

- **Previous Owners**:
  - Should be integer but contains float values
  - Right-skewed distribution as expected
  - Maximum values need validation (unusually high number of previous owners)

<a id="pre-processing"></a>
## Pre-processing

<a id="functions"></a>
### Functions

In [ ]:
def  general_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    """Perform general data cleaning on the DataFrame.
    
    This function handles logical inconsistencies and data quality issues that
    don't require statistical calculations (mean, median, etc.) to prevent data
    leakage between training and validation sets.
    
    Args:
        df (pd.DataFrame): The input DataFrame containing car data with columns:
            Brand, model, year, transmission, fuelType, mileage, tax, mpg, 
            engineSize, paintQuality%, previousOwners, hasDamage.
        
    Returns:
        pd.DataFrame: The cleaned DataFrame with logical inconsistencies resolved.
    """

    df = df.copy()

    # Set negative values to NaN for features that shouldn't be negative
    for col in ['previousOwners', 'mileage', 'tax', 'mpg', 'engineSize']:
        df.loc[df[col] < 0, col] = np.nan

    for col in ['Brand', 'model', 'transmission', 'fuelType']:
        df[col] = df[col].str.lower()
        df[col] = df[col].replace('', np.nan)

    # Remove decimal part from 'year'
    df['year'] = np.floor(df['year'])

    # Remove decimal part from 'previousOwners'
    df['previousOwners'] = np.floor(df['previousOwners'])

    # Ensure 'paintQuality%' is within 0-100
    df.loc[(df['paintQuality%'] < 0) | (df['paintQuality%'] > 100), 'paintQuality%'] = np.nan

    return df

In [ ]:
def standardize_categorical_col(series: pd.Series, standardised_cats: list[str]) -> pd.Series:
    """Standardizes the entries of a categorical column based on a list of standard categories.
    The function uses edit distance to find the closest match from the standard categories.
    for example, "Vokswagen" would be standardized to "Volkswagen".
    
    Args:
        series (pd.Series): The categorical column to standardize.
        standardised_cats (list[str]): List of standard category names.
        
    Returns:
        pd.Series: The standardized categorical column.
    """
    
    def find_closest_match(x):
        if pd.isnull(x):
            return np.nan
        distances = [nltk.edit_distance(str(x), cat) for cat in standardised_cats]
        return standardised_cats[np.argmin(distances)]
    
    return series.apply(find_closest_match)


In [ ]:
def get_categories_high_freq(series: pd.Series, threshold: int = 100) -> list[str]:
    """Get categories in a categorical column that appear more than a specified threshold.
    
    Args:
        series (pd.Series): The categorical series to analyze.
        threshold (int): The minimum frequency for a category to be included.
        
    Returns:
        list[str]: List of categories with frequency above the threshold.
    """
    value_counts = series.value_counts()
    high_freq_cats = value_counts[value_counts > threshold].index.tolist()
    return high_freq_cats


In [ ]:
X = df.drop(columns=["price"])   
y = df["price"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")

In [ ]:
X_train_clean = X_train.copy()
X_val_clean = X_val.copy()

<a id="missing-values"></a>
### Missing Values

In [ ]:
num_general = ["tax", "previousOwners", "paintQuality%", "engineSize"]

median_imputer = SimpleImputer(strategy="median")
X_train_clean[num_general] = median_imputer.fit_transform(X_train_clean[num_general])
X_val_clean[num_general] = median_imputer.transform(X_val_clean[num_general])

In [ ]:
if "mpg" in X_train_clean.columns and "fuelType" in X_train_clean.columns:
    mpg_group_medians = X_train_clean.groupby("fuelType")["mpg"].median()

    def impute_mpg(row):
        if pd.isnull(row["mpg"]):
            return mpg_group_medians.get(row["fuelType"], X_train_clean["mpg"].median())
        return row["mpg"]

    X_train_clean["mpg"] = X_train_clean.apply(impute_mpg, axis=1)
    X_val_clean["mpg"] = X_val_clean.apply(impute_mpg, axis=1)

In [ ]:
missing_summary_train = X_train_clean.isnull().sum()
missing_summary_val = X_val_clean.isnull().sum()

print("Remaining missing values (train):", missing_summary_train.sum())
print("Remaining missing values (validation):", missing_summary_val.sum())

In [ ]:
remaining_missing = X_train_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print("Columns with remaining NaN values:")
display(remaining_missing)

In [ ]:
remaining_missing = X_train_clean.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0]
print("Columns with remaining NaN values:")
display(remaining_missing)

### Numeric Features

- carID
- year
- price
- mileage
- tax
- mpg
- engineSize
- paintQuality
- previousOwners
- hasDamage


In [ ]:
#Check if there are negative features that should not be negative
numeric_features = X_train_clean.select_dtypes(include=['int64', 'float64']).columns
negative_values = {}
for col in numeric_features:
    negative_count = (X_train_clean[col] < 0).sum()
    if negative_count > 0:
        negative_values[col] = negative_count   

for v in negative_values:
    print(f"Feature '{v}' has {negative_values[v]} negative values.")

### Strategy:

- Change negative values to `NaN`
- Remove extreme outliers
- Impute using the appropriate method (median, mode, or group-based)
- Convert the column back to integer type if applicable

Note: Still have to choose what method to use to change the missing values in the feature "hasDamage" before I can switch from float to int

In [ ]:
cols_to_fix = list(negative_values.keys())

for feature in cols_to_fix:
    X_train_clean.loc[X_train_clean[feature] < 0, feature] = np.nan
    X_val_clean.loc[X_val_clean[feature] < 0, feature] = np.nan


In [ ]:
for col in X_train_clean.select_dtypes(include=['int64', 'float64']).columns:
    q1 = X_train_clean[col].quantile(0.25)
    q3 = X_train_clean[col].quantile(0.75)
    iqr = q3 - q1
    lower_lim = q1 - (1.5 * iqr)
    upper_lim = q3 + (1.5 * iqr)
    X_train_clean[col] = X_train_clean[col].mask((X_train_clean[col] < lower_lim) | (X_train_clean[col] > upper_lim), np.nan)
    X_val_clean[col] = X_val_clean[col].mask((X_val_clean[col] < lower_lim) | (X_val_clean[col] > upper_lim), np.nan)


In [ ]:
for feature in cols_to_fix:
    median_value = X_train_clean[feature].median()
    X_train_clean[feature].fillna(median_value, inplace=True)
    X_val_clean[feature].fillna(median_value, inplace=True)


In [ ]:
int_cols = ['year', 'previousOwners']

median_year = X_train_clean['year'].median()
X_train_clean['year'].fillna(median_year, inplace=True)
X_val_clean['year'].fillna(median_year, inplace=True)

for feature in int_cols:
    X_train_clean[feature] = X_train_clean[feature].astype(int)
    X_val_clean[feature] = X_val_clean[feature].astype(int)

In [ ]:
(X_train_clean["year"] < 1950).sum() and (X_train_clean["year"] > 2025).sum()

Final check for nan values

Check for other strange values

In [ ]:
X_train_clean.loc[(X_train_clean['paintQuality%'] > 100), 'paintQuality%'] = np.nan
X_val_clean.loc[(X_val_clean['paintQuality%'] > 100), 'paintQuality%'] = np.nan

median_paint = X_train_clean['paintQuality%'].median()
X_train_clean['paintQuality%'].fillna(median_paint, inplace=True)
X_val_clean['paintQuality%'].fillna(median_paint, inplace=True)



In [ ]:
missing_counts = X_train_clean.isnull().sum()
missing_percent = (missing_counts / len(df)) * 100

missing_summary = pd.DataFrame({
    "Missing Count": missing_counts,
    "Missing %": missing_percent.round(2)
}).sort_values(by="Missing %", ascending=False)

missing_summary[missing_summary["Missing Count"] > 0]

In [ ]:
num_cols = X_train_clean.select_dtypes(include=["int64", "float64"]).columns

outlier_summary = []

for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = X_train_clean[(X_train_clean[col] < lower_bound) | (X_train_clean[col] > upper_bound)]
    
    outlier_summary.append({
        "Feature": col,
        "Lower Bound": round(lower_bound, 2),
        "Upper Bound": round(upper_bound, 2),
        "Outlier Count": len(outliers),
        "Outlier %": round(len(outliers) / len(df) * 100, 2)
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values(by="Outlier %", ascending=False)
outlier_df